# NLP Assignment: Language Modelling with Poetry

This assignment will check your basic NLP understanding through the fundamental NLP task of **language modelling**. 

You will reiterate on the task with techniques ranging from simple n-gram counting to embeddings and deep learning.


You can work with the same short poetry texts that should be familiar to you by now:

Unless stated otherwise (Ex. 4,5,6), work just with the Robert Frost file for simplicity (Ex. 1,2,3,7,8)

In [1]:
from abc import ABC, abstractmethod

import numpy as np
import plotly.express as px
from sklearn.decomposition import TruncatedSVD


rng = np.random.default_rng()

#### **Excercise 1**: Markov Language Model (**2 points**)

1. Create a 1st order Markov (bi-gram) language model 
  - work with matrix representation of the model
    - i.e. not dictionaries as we did in the tutorial!
    - hence you'll need to assign indices to the words, too
      - include an extra \<UNK\> token for unseen words
  - correctly handle beginnings and ends of sentences
    - sentence = line
  - lower case and properly tokenize your sentences
    - but you may skip all extra text preprocessing

In [2]:
class Model(ABC):
    @abstractmethod
    def create_model(self, corpus: list):
        pass


class BiGramModel(Model):
    """
    1st order Markov (bi-gram) language model.
    """
    def __init__(self, vocabulary: set, corpus: list, smooth: bool):
        self.vocabulary = vocabulary

        self.model = np.zeros([len(vocabulary) + 1, len(vocabulary) + 1])  # +1 for <UNK>

        self.token_to_idx = {}
        self.idx_to_token = {}
        for idx, token in enumerate(vocabulary.union(["<UNK>"])):
            self.token_to_idx[token] = idx
            self.idx_to_token[idx] = token

        self.create_model(corpus, smooth)

    def create_model(self, corpus: list, smooth: bool):
        for tokens in corpus:
            for i in range(1, len(tokens)):
                self.model[self.token_to_idx[tokens[i - 1]], self.token_to_idx[tokens[i]]] += 1

        if smooth:
            self.model_by_laplace_estimate(corpus)
        else:
            self.model_by_maximum_likelihood(corpus)
            self.model[np.isnan(self.model)] = 0

        assert np.sum(np.isnan(self.model)) == 0

    def model_by_maximum_likelihood(self, corpus: list):
        self.model /= np.sum(self.model, axis=1).reshape(-1, 1)

    def model_by_laplace_estimate(self, corpus: list):
        self.model += 1
        self.model /= np.sum(self.model, axis=1).reshape(-1, 1)

    def get_token_idx(self, token):
        if self.token_to_idx.get(token) is None:
            return self.token_to_idx["<UNK>"]
        else:
            return self.token_to_idx[token]


def assert_marginals_equal_one(model: Model, smooth: bool): 
    for i in range(model.model.shape[0]):

        if smooth is False:
            if model.idx_to_token[i] == "<UNK>" or model.idx_to_token[i] == "</s>":
                continue

        prob = np.sum(model.model[i, :])
        assert np.abs(prob - 1) < 10**-9, f"prob={prob} at token: \"{model.idx_to_token[i]}\""


In [3]:
def get_vocabulary_and_corpus(path: str):
    vocabulary = set()
    corpus = []

    for line in open(path, encoding='utf-8'):
        tokens = line.rstrip().lower().split()
        if not tokens:
            continue
        tokens = ['<s>'] + tokens + ['</s>']
        vocabulary.update(tokens)
        corpus.append(tokens)

    return vocabulary, corpus

In [4]:
!wget -nc https://raw.githubusercontent.com/GustikS/smu-nlp/master/robert_frost.txt
!wget -nc https://raw.githubusercontent.com/GustikS/smu-nlp/master/edgar_allan_poe.txt

vocabulary_frost, corpus_frost = get_vocabulary_and_corpus("robert_frost.txt")
vocabulary_poe, corpus_poe = get_vocabulary_and_corpus("edgar_allan_poe.txt")

# Exercise 1. Create bi-gram model.
smooth = True
model_frost = BiGramModel(vocabulary_frost, corpus_frost, smooth=smooth)
assert_marginals_equal_one(model_frost, smooth)


--2022-06-06 17:40:15--  https://raw.githubusercontent.com/GustikS/smu-nlp/master/robert_frost.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 56286 (55K) [text/plain]
Saving to: ‘robert_frost.txt’

robert_frost.txt    100%[===================>]  54.97K  --.-KB/s    in 0.008s  

2022-06-06 17:40:16 (6.91 MB/s) - ‘robert_frost.txt’ saved [56286/56286]

--2022-06-06 17:40:16--  https://raw.githubusercontent.com/GustikS/smu-nlp/master/edgar_allan_poe.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26622 (26K) [text/plain]
Sav

#### **Excercise 2**: Probability + Smoothing (**1 points**)
1. write a function to obtain probability of a given sentence with your model
    - include the beginning and end mark of the sentence as well
2. incorporate the add-1 (Laplace) smoothing to account for unseen bi-grams
    - you should have your \<UNK\> for unseen unigrams already

In [5]:
def get_log_probability(sentence: list, model: BiGramModel) -> float:
    prob_log = 0
    n = len(sentence)
    for i in range(1, n):
        idx1 = model.get_token_idx(sentence[i - 1])
        idx2 = model.get_token_idx(sentence[i])
        p = np.log(model.model[idx1, idx2])
        prob_log += p
    return prob_log


def get_probability(sentence: list, model: BiGramModel) -> float:
    prob = 1
    n = len(sentence)
    for i in range(1, n):
        idx1 = model.get_token_idx(sentence[i - 1])
        idx2 = model.get_token_idx(sentence[i])
        p = model.model[idx1, idx2]
        prob *= p
    return prob

In [6]:
sentence = corpus_frost[rng.integers(0, len(corpus_frost))] 
print("==== Exercise 2. ====")
print(f"Probability of \"{' '.join(sentence[1:-1])}\" is {get_probability(sentence, model_frost)}.")

# sentence = "<s> the young folk held some hope out to each other. </s>".split()
# print(f"Probability of \"{' '.join(sentence[1:-1])}\" is {get_probability(sentence, model_frost)}.")

==== Exercise 2. ====
Probability of "the least i could do was to help dig their grave." is 1.3656047530533152e-36.


#### **Excercise 3**: Perplexity (**1 points**)
1. write a function fo calculate perplexity of your smoothed model on a given sentence
  - including its beginning and ending
2. find the sentence(s) from the corpus with minimum and maximum perplexity

In [7]:
def get_perplexity(sentence: list, model: Model) -> float:
    n = len(sentence)
    perp = 1
    for i in range(1, n):
        idx1 = model.get_token_idx(sentence[i - 1])
        idx2 = model.get_token_idx(sentence[i])
        perp *= 1 / model.model[idx1, idx2]
    return np.power(perp, 1 / n)


In [8]:
if not smooth:
    sentence = "the young folk held some hope out to each other.".split()
    print(f'{get_perplexity(sentence, model_frost)}')

max_perplexity = 0
min_perplexity = np.inf
min_idx = np.nan
max_idx = np.nan
for i, sentence in enumerate(corpus_frost):
    perp = get_perplexity(sentence, model_frost)
    if perp > max_perplexity:
        max_perplexity = perp
        max_idx = i
    if perp < min_perplexity:
        min_perplexity = perp
        min_idx = i

print("\n==== Exercise 3. ====")
print(f"Sentence with minimum perplexity = {np.round(min_perplexity, 2)}: \"{' '.join(corpus_frost[min_idx][1:-1])}\"")
print(f"Sentence with maximum perplexity = {np.round(max_perplexity, 2)}: \"{' '.join(corpus_frost[max_idx][1:-1])}\"")


==== Exercise 3. ====
Sentence with minimum perplexity = 75.59: "it."
Sentence with maximum perplexity = 843.08: "life long in bed, and wrote her things in bed."


#### **Excercise 4:**  Markov classifier (**4 points**)
Implement your own probabilistic classifier based on your bi-gram language model. That is:
  1. given some classes of sentences, train a separate language model for each class respectively
  2. then classify a given sentence (=sample) based on maximum a-posteriori probability (MAP)
    - i.e. don't forget about the class priors, too!
    - use log probabilities!
    - make sure your smoothing treats all the classes equally!
  3. evaluate on a task of recognizing sentences from the 2 different poets (Frost vs. Poe)
    - split the sentences (=samples) from each poet into train-test in advance!
      - skip empty lines (of course)

*Note that this is very similar to our previous classification with Naive Bayes, but with bi-grams instead of unigrams.*

In [9]:
class MarkovClassifier:
    """
    Naive Bayes classifier using Markov models.
    """
    def __init__(self, prior_1: int, prior_2: int, 
                 model_1: Model, model_2: Model,
                 name_class_1=None, name_class_2=None):
        self.prior_1 = prior_1
        self.prior_2 = prior_2
        self.model_1 = model_1
        self.model_2 = model_2
        self.name_class_1 = name_class_1
        self.name_class_2 = name_class_2

    def classify(self, sentence: list):
        p_1 = np.log(self.prior_1) + get_log_probability(sentence, self.model_1)
        p_2 = np.log(self.prior_2) + get_log_probability(sentence, self.model_2)

        if self.name_class_1 is not None:
            return self.name_class_1 if p_1 > p_2 else self.name_class_2
        else:
            return 1 if p_1 > p_2 else 2


def get_train_test_indices(length, train_fraction=0.8):
    indices = rng.permutation(np.arange(length))
    delim = np.round(length * train_fraction).astype(np.int32)
    return indices[0: delim], indices[delim:]

In [10]:
vocabulary = vocabulary_frost.union(vocabulary_poe)

frost_idcs_train, frost_idcs_test = get_train_test_indices(len(corpus_frost))
poe_idcs_train, poe_idcs_test = get_train_test_indices(len(corpus_poe))
corpus_frost_train = [corpus_frost[idx] for idx in frost_idcs_train]
corpus_frost_test = [corpus_frost[idx] for idx in frost_idcs_test]
corpus_poe_train = [corpus_poe[idx] for idx in poe_idcs_train]
corpus_poe_test = [corpus_poe[idx] for idx in poe_idcs_test]

# # The two vocabularies are presumably not identical. 
# # The smoothing is done for each vocabulary independently.
# # The probability of <UNK> is 1/V differs. 
# # So I will use V as union of both vocabularies.
# print(np.sum(model_poe.model[model_poe.token_to_idx["<UNK>"], :]))
# print(np.sum(model_frost.model[model_frost.token_to_idx["<UNK>"], :]))
# Training on the different corpora which can differ in size,
#   but on union of vocabularies, thus smoothing treat classes equally.

model_frost = BiGramModel(vocabulary, corpus_frost_train, smooth=True)
model_poe = BiGramModel(vocabulary, corpus_poe_train, smooth=True)

# assert marginals == 1
for i in range(model_frost.model.shape[0]):
    prob = np.sum(model_frost.model[i, :])
    assert np.abs(prob - 1) < 10**-9, f"prob={prob} at token: \"{model_frost.idx_to_token[i]}\""

# number_of_tokens_frost = 0
# number_of_tokens_poe = 0
# for sentence in corpus_frost:
#     number_of_tokens_frost += len(sentence) - 2
# for sentence in corpus_poe:
#     number_of_tokens_poe += len(sentence) - 2
# number_of_tokens = (number_of_tokens_poe + number_of_tokens_frost)
# prior_poe = number_of_tokens_poe / number_of_tokens
# prior_frost = number_of_tokens_frost / number_of_tokens

# As samples are sentences, I calculate priors rather from 
#  their respective sizes, rather than by tokens.
corpus = corpus_frost + corpus_poe
prior_poe = len(corpus_poe) / len(corpus)
prior_frost = len(corpus_frost) / len(corpus)

print("\n==== Exercise 4. ====")
print(f"prior_frost={np.round(prior_frost, 2)}")
print(f"prior_poe={np.round(prior_poe, 2)}")

markov_classifier = MarkovClassifier(prior_1=prior_poe, prior_2=prior_frost,
                                      model_1=model_poe, model_2=model_frost,
                                      name_class_1="poe", name_class_2="frost")

correctly_classified_poe = 0
for sentence in corpus_poe_test:
    if markov_classifier.classify(sentence) == "poe":
        correctly_classified_poe += 1

correctly_classified_frost = 0
for sentence in corpus_frost_test:
    if markov_classifier.classify(sentence) == "frost":
        correctly_classified_frost += 1

test_accuracy_poe = correctly_classified_poe / len(corpus_poe_test)
print(f"Test accuracy Poe: {np.round(test_accuracy_poe * 100, 2)}%.")

test_accuracy_frost = correctly_classified_frost / len(corpus_frost_test)
print(f"Test accuracy Frost: {np.round(test_accuracy_frost * 100, 2)}%.")


==== Exercise 4. ====
prior_frost=0.67
prior_poe=0.33
Test accuracy Poe: 50.69%.
Test accuracy Frost: 95.12%.


#### **Excercise 5**: PPMI word-word cooccurence (**2 points**)
1. Create a word-word co-occurence matrix from all the text of the both poets altogether
  - use a sliding window of size 5 (2 left + 2 right context words)
    - remember that you can reuse existing solutions...
2. Post-process the matrix with PPMI

In [11]:
class TokenIndexer:
    def __init__(self, vocabulary, include_unk):
        self.token_to_idx = {}
        self.idx_to_token = {}

        unk = "<UNK>" if include_unk else ""
        for idx, token in enumerate(vocabulary.union(unk)):
            self.token_to_idx[token] = idx
            self.idx_to_token[idx] = token

    def get_token_idx(self, token):
        if self.token_to_idx.get(token) is None:
            return self.token_to_idx["<UNK>"]
        else:
            return self.token_to_idx[token]

    def get_idx_token(self, idx):
        if self.idx_to_token.get(idx) is None:
            return self.idx_to_token["<UNK>"]
        else:
            return self.idx_to_token[idx]


class PPMI:
    def __init__(self, corpus: list, vocabulary: set, 
                 context_size: int, token_indexer: TokenIndexer):
        self.text = []
        self.get_text(corpus)

        self.vocabulary = vocabulary
        self.token_indexer = token_indexer
        self.context_size = context_size
        self.half_window_size = (context_size - 1) // 2

        self.co_occurrence_matrix = np.zeros([len(vocabulary), len(vocabulary)], dtype=np.int32)
        self.populate_co_occurrence_matrix()

        self.ppmi_matrix = np.zeros([len(vocabulary), len(vocabulary)])
        self.create_ppmi_matrix()

    def get_text(self, corpus):
        for line in corpus:
            self.text += line[1: -1]  # <s> and </s>

    def populate_co_occurrence_matrix(self):
        for i in range(self.half_window_size, len(self.text) - self.half_window_size):
            context_tokens = self.text[i - self.half_window_size: i] + self.text[i + 1: i + 1 + self.half_window_size]
            idx1 = self.token_indexer.get_token_idx(self.text[i])
            for token in context_tokens:
                idx2 = self.token_indexer.get_token_idx(token)
                self.co_occurrence_matrix[idx1, idx2] += 1
                self.co_occurrence_matrix[idx2, idx1] += 1

    def create_ppmi_matrix(self):
        rows_sums = np.sum(self.co_occurrence_matrix, axis=0)
        cols_sums = np.sum(self.co_occurrence_matrix, axis=1)
        total_sum = np.sum(self.co_occurrence_matrix)
        expected = np.outer(rows_sums, cols_sums) / total_sum
        self.ppmi_matrix = np.log(self.co_occurrence_matrix / expected)
        self.ppmi_matrix[np.isnan(self.ppmi_matrix)] = 0
        self.ppmi_matrix[self.ppmi_matrix < 0] = 0


In [12]:
token_indexer = TokenIndexer(vocabulary, include_unk=False)

ppmi = PPMI(corpus=corpus_frost + corpus_poe, 
            vocabulary=vocabulary, 
            context_size=5, 
            token_indexer=token_indexer)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:59: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:59: RuntimeWarning: divide by zero encountered in log


#### **Excercise 6**: Word embeddings (**1 points**)
1. Extract 8-dimensional word embeddings from your PPMI matrix
  - using the truncated SVD matrix decomposition
2. Plot them in 2D with word labels

In [13]:
components = TruncatedSVD(n_components=8).fit_transform(ppmi.ppmi_matrix)
fig = px.scatter(x=components[:, 0], y=components[:, 1], text=list(token_indexer.token_to_idx.keys()),
                  size_max=60, title="first two SVD components of PPMI matrix.")
fig.update_traces(textposition='top center')
fig.show()

#### **Excercise 7:** LSTM Language Model (**3 points**)
1. Formulate a proper dataset for language modelling on longer sequences from the text
  - beyond the n-gram scope, e.g. 10 consecutive words
2. Create a suitable LSTM-based language model
3. Initialize the embedding layer of your model with your "pretrained" SVD embeddings

In [20]:
import numpy as np

import pandas as pd

import nltk
from nltk import word_tokenize
nltk.download('punkt')

import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
torch.device('cuda' if torch.cuda.is_available() else 'cpu')


class ModelLSTM(nn.Module):
    def __init__(self, vocab_size, sequence_len, pretrained_vectors: np.ndarray=None):
        super(ModelLSTM, self).__init__()

        self.hidden_dim = 64
        self.num_layers = 16
        self.sequence_len = sequence_len

        if pretrained_vectors is None:
            self.embedding_dim = 32
            self.embedding = torch.nn.Embedding(vocab_size, self.embedding_dim)
        else:
            self.embedding_dim = pretrained_vectors.shape[1]
            self.embedding = torch.nn.Embedding.from_pretrained(
                torch.from_numpy(pretrained_vectors).float(), freeze=False)

        self.h_0, self.c_0 = self.init_hidden(self.sequence_len)
        self.rnn = torch.nn.LSTM(self.embedding_dim, self.hidden_dim, self.num_layers)

        self.fc = torch.nn.Linear(self.hidden_dim, vocab_size)

    def init_hidden(self, sequence_length):
        return (torch.autograd.Variable(torch.randn(self.num_layers, sequence_length, self.hidden_dim)),
                torch.autograd.Variable(torch.randn(self.num_layers, sequence_length, self.hidden_dim)))


    def forward(self, x):
        embed = self.embedding(x)

        output, (self.h_0, self.c_0) = self.rnn(embed, (self.h_0, self.c_0))

        logits = self.fc(output)
        return logits


class DatasetPoetrySequential(torch.utils.data.Dataset):
    def __init__(self, lines, word2idx, sequence_length=10):
        self.sequence_length = sequence_length
        self.words = [item for sublist in lines for item in sublist]  # Creates text as list
        self.words_indices = [word2idx[w] for w in self.words]  #  with tokens as indices.

    def __len__(self):
        return len(self.words_indices) - self.sequence_length

    def __getitem__(self, index):
        return (
            torch.tensor(self.words_indices[index:index + self.sequence_length]),
            torch.tensor(self.words_indices[index + 1:index + self.sequence_length + 1]))  # the same sequence shifted by 1 step


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [21]:
!wget -nc https://raw.githubusercontent.com/GustikS/smu-nlp/master/robert_frost.txt

lines = []  
word2idx = {}
idx2word = []  
idx = 0
for line in open('robert_frost.txt'):
  tokens = line.rstrip().lower().split()
  if not tokens:
    continue
  lines.append(tokens)
  tokens = ['<ss>', '<s>'] + tokens + ['</s>']
  for i in range(0, len(tokens)): 
    if tokens[i] not in word2idx:
      word2idx[tokens[i]] = idx
      idx2word.append(tokens[i])
      idx += 1


sequence_length = 10
batch_size = 2048
epochs = 3

poetry_data = DatasetPoetrySequential(lines, word2idx)

dataloader = DataLoader(poetry_data, batch_size=batch_size)

model = ModelLSTM(vocab_size=len(word2idx), 
                  sequence_len=sequence_length,
                  pretrained_vectors=components)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

for epoch in range(epochs):
    print(f"epoch {epoch}: ", end='')
    model.train()

    loss_train = 0.0
    correct = 0
    train_data_size = 0
    for i, (x, y) in enumerate(dataloader):

        optimizer.zero_grad()
        # model.h_0.detach()
        # model.c_0.detach()
        model.h_0, model.c_0 = model.init_hidden(sequence_length)

        y_pred = model(x)  

        loss = criterion(y_pred.transpose(1, 2), y)

        loss.backward()
        optimizer.step()
        
        loss_train += loss.item()

        max_vals, max_idcs = torch.max(y_pred, dim=2)
        # print(y_pred.shape)
        # print(y.shape)
        # print(y)
        # print(y_pred)
        # print(max_vals.shape)
        # print(max_idcs)
        
        correct += (max_idcs == y).float().sum()
        train_data_size += y.shape[0]
        print("==", end='')

    acc_train = 100 * correct / train_data_size
    print(f'  || training loss: {np.round(loss_train, 3)}  | training accuracy: {np.round(acc_train.detach().numpy(), 3)} %')

# notes https://jamesmccaffrey.wordpress.com/2021/04/14/explaining-the-pytorch-embeddingbag-layer/

File ‘robert_frost.txt’ already there; not retrieving.

epoch 0: ============  || training loss: 45.66  | training accuracy: 31.770999908447266 %
epoch 1: ============  || training loss: 40.128  | training accuracy: 50.191001892089844 %
epoch 2: ============  || training loss: 39.788  | training accuracy: 31.892000198364258 %


In [16]:
print(len(idx2word))
print(len(vocabulary_frost))

print(vocabulary_frost.difference(set(idx2word)))
print(set(idx2word).difference(vocabulary_frost))

3057
3056
set()
{'<ss>'}


 #### **Excercise 8**: Sampling (**1 point**)
1. Sample some text from your models w.r.t. to the output (next) word probability distribution
  - for both bigram and LSTM models - which is better?

2. Paste your best "poem" here: 

In [22]:
def sample_word(probabilities: np.ndarray):
    p_threshold = np.sum(probabilities) * rng.random()  # Unif[a,b): (b - a) * random() + a,
    p_cumulative = 0
    for idx in range(len(probabilities)):
        p_cumulative += probabilities[idx]
        if p_threshold < p_cumulative:
            return idx
    assert False, f"p_cumulative={p_cumulative}, p_thr={p_threshold}"


def sample_from_model(model: Model, number_of_sentences=4):
    for _ in range(number_of_sentences):
        sentence = []
        idx = model.get_token_idx("<s>")
        for i in range(50):  # Generating on CzechVerseCorpus with smoothed model with vocab ~10**5 is hard to hit <\s>
            next_idx = sample_word(model.model[idx, :])
            next_word = model.idx_to_token[next_idx]
            if next_word == "</s>":
                break
            sentence.append(next_word)
            idx = next_idx
        print(' '.join(sentence))

def sample_from_model_top_p(model: Model, number_of_sentences=4, top_p=5):
    for _ in range(number_of_sentences):
        sentence = []
        idx = model.get_token_idx("<s>")
        for i in range(50):
            probs = model.model[idx, :]
            idcs = np.argpartition(probs, -top_p)[-top_p:]
            x = probs[idcs]
            idx_to_partition = sample_word(probs[idcs])
            next_idx = idcs[idx_to_partition]
            next_word = model.idx_to_token[next_idx]
            if next_word == "</s>":
                print(' '.join(sentence))
                break
            sentence.append(next_word)
            idx = next_idx


def sample_from_torch_model(model, corpus, number_of_sentences=4):
    model.eval()
    sentences_seeds = [] 

    for _ in range(number_of_sentences):
      words = corpus[rng.integers(0, len(corpus))]
      sentences_seeds.append(words)
      words = words[1: len(words) // 3]
      model.h_0, model.c_0 = model.init_hidden(len(words))

      for i in range(0, rng.integers(0, 20)):
          x = torch.tensor([[word2idx[w] for w in words[i:]]])
          y_pred = model(x)
          last_word_logits = y_pred[0][-1]
          p = torch.nn.functional.softmax(last_word_logits, dim=0).detach().numpy()
          word_index = np.random.choice(len(last_word_logits), p=p)
          words.append(idx2word[word_index])
      print(' '.join(words))

    print("\n(Sentence from which seed is created by taking first third of sentence, for comparison.)")
    for seed in sentences_seeds:
        print(' '.join(seed[1:-1]))


In [23]:
print("\n==== Exercise 8. ====")

print("*Sample from not-smoothed bi-gram model trained on Frost corpus:")
sample_from_model(BiGramModel(vocabulary_frost, corpus_frost, smooth=False))

top_p = 50
print(f"\n*Sample from not-smoothed bi-gram model trained on Frost corpus, best {top_p} words:")
sample_from_model_top_p(BiGramModel(vocabulary_frost, corpus_frost, smooth=False), top_p=top_p)


print(f"\n*Sample from LSTM model.")
sample_from_torch_model(model, corpus_frost)


==== Exercise 8. ====
*Sample from not-smoothed bi-gram model trained on Frost corpus:
so's the flowers.
of the job.'
(the kitchen to do to myself to say,
cold

*Sample from not-smoothed bi-gram model trained on Frost corpus, best 50 words:
we wouldn't show.
what is tending.
of having interfered in the one fresh and shouted, 'shut in' for her straight down from him throw a tree that as fast
she had it doesn't seem so far as if

*Sample from LSTM model.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: RuntimeWarning:

invalid value encountered in true_divide



a friend or panther heart miles town tempt to at 'from like there on began it's know flowers said every was
in warren bedroom south in meeting had him. it's needn't
a barkless watched eye a seems with and i'm prove in for his not me
here he melt let else. and on together. and battle and young

(Sentence from which seed is created by taking first third of sentence, for comparison.)
a friend or two as good as own the farm,
in warren at march meeting for the reason
a barkless spectre. he had halted too,
here he is now. this box! put it away.


# **Notes**

If this seems like a lot of work, reiterate through the tutorials (which is the purpose, anyway) - we did most of this already!
  - i.e., I don't expect you to be overly creative, you can find all the pieces in the tutorial notebooks
  - but feel free to write a more efficient/clean/suitable code from scratch
  - only the libraries we used in the tutorials are allowed
  - teamwork is forbidden!
  - make sure your code is runnable (in Colab) before submitting!


I will only evaluate correctness of the solutions, not quality of the models
  - i.e. no need to spent too much time with text preprocessing or hyperparameter tuning (LSTM)

# **Bonus points**
You can recover some of the potentially lost points here, but you can't reach more than 15 points in total from this assignment

1. (1p) Use a 2nd order Markov model with a sparse 3D array instead
  - i.e., you can do Laplace smoothing without actually modifying the model (tensor)!

2. (1p) In your LSTM model, do not start every sample sequence with an empty hidden state (zeros), but pass the last hidden state from the preceding sample sequence
  - i.e. the sequence ending just before the start of the current sequence
  - but do not propagate gradient into the previous sample (detach the state value from the backprop graph)

3. (1p) Improve your text generation by sampling from the top-p next words only

4. (2p) Use a different (larger) textual corpus of poems
  - e.g. Corpus of Czech Verse, collected at the Institute of Czech Literature at Czech Academy of Sciences
    - available at https://github.com/versotym/corpusCzechVerse
    - just be careful about czech character encoding


**Bonus task 1.**

Scipy doesn't offer interface for dimensions higher than two (to my best knowledge), which I find quite surprising.

I have to either:
1. use other package (which is forbidden in this hw)
2. implement my own using python built-ins (eg.dictionary)
3. use sparse matrix from scipy in shape of [n * n, n] where n is the vocabulary size.



In [24]:
import scipy.sparse


class TriGramModel(Model):
    """
    2nd order Markov (tri-gram) language model.
    """

    def __init__(self, vocabulary: set, corpus: list, smooth: bool):
        self.vocabulary = vocabulary

        self.model = scipy.sparse.lil_matrix((len(vocabulary) * len(vocabulary), len(vocabulary)), dtype=np.int8)
        self.smooth = smooth

        self.token_to_idx = {}
        self.idx_to_token = {}
        for idx, token in enumerate(vocabulary):
            if token == "<UNK>":
                continue
            self.token_to_idx[token] = idx
            self.idx_to_token[idx] = token

        self.create_model(corpus, smooth)

    def create_model(self, corpus: list, smooth: bool):
        for tokens in corpus:
            for i in range(2, len(tokens)):
                idx1 = self.token_to_idx[tokens[i - 2]]
                idx2 = self.token_to_idx[tokens[i - 1]]
                idx3 = self.token_to_idx[tokens[i]]
                self.model[idx1 * len(self.vocabulary) + idx2, idx3] += 1
        self.model_by_maximum_likelihood(corpus)

    def model_by_maximum_likelihood(self, corpus: list):
        cols_sums = self.model.sum(axis=1)
        x = np.where(cols_sums > 0)
        self.model[x] = self.model[x] / cols_sums[x]

    def get_prob(self, i, j, k):
        if self.smooth:
            cols_sum = self.model[i * len(self.vocabulary) + j, :].sum()
            return (self.model[i * len(self.vocabulary) + j, k] + 1) / (cols_sum + len(self.vocabulary))
        else:
            return self.model[i * len(self.vocabulary) + j, k]

    def get_token_idx(self, token):
        if self.token_to_idx.get(token) is None:
            return self.token_to_idx["<UNK>"]
        else:
            return self.token_to_idx[token]


def get_vocabulary_and_corpus_trigram(path: str):
    vocabulary = set()
    corpus = []

    for line in open(path, encoding='utf-8'):
        tokens = line.rstrip().lower().split()
        if not tokens:
            continue
        tokens = ['<ss>'] + ['<s>'] + tokens + ['</s>']
        vocabulary.update(tokens)
        corpus.append(tokens)

    return vocabulary, corpus

In [27]:
v, c = get_vocabulary_and_corpus_trigram("robert_frost.txt")
v = v.union("<UNK>")
model_frost = TriGramModel(v, c, smooth=True)

In [28]:
idx1 = model_frost.get_token_idx('well')
idx2 = model_frost.get_token_idx('may')
idx3 = model_frost.get_token_idx('prove')
print(model_frost.get_prob(idx1, idx2, idx3))

if model_frost.smooth:
  assert model_frost.model[idx1 * len(v) + idx2, idx3] == 1

0.000652954619653934


**Bonus task 2.**

implemented directly in Exercise 7. 

I do not detach the hidden states, which I was unable to make work (in every batch model.h_0.detach(), model.c_0.detach())
so as work around, I initialize hidden state with new Variable every batch, which has the same effect, (could be less efficient though).

**Bonus task 3.**

implemented directly in Exercise 8.

**Bonus task 4.**

I am unable to load the whole CzechVerseCorpus (RAM crash), even with 50 books using BiGramModel implemented by numpy matrices crashes ram, so only portion *number_of_books*=20 is taken.

In [29]:
!git clone https://github.com/versotym/corpusCzechVerse

import json
from os import listdir
path = 'corpusCzechVerse/ccv/'

number_of_books = 20

books = []
files = [f for f in listdir(path)]
for i, f in enumerate(sorted(files)):
    if i > number_of_books:
        break
    book = open(path + f)
    books.append(json.load(book))


Cloning into 'corpusCzechVerse'...
remote: Enumerating objects: 1324, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 1324 (delta 3), reused 0 (delta 0), pack-reused 1312
Receiving objects: 100% (1324/1324), 278.87 MiB | 24.65 MiB/s, done.
Resolving deltas: 100% (1301/1301), done.
Checking out files: 100% (1306/1306), done.


In [30]:
# i=0
# book = books[i]

# for poem in book:
#     author = poem['p_author']['name']
#     title = poem['biblio']['p_title']
#     print('------' + title + '------\n')
#     body = poem['body']
#     for text in body:
#         for line in text:
#             print(line['text'])
#         print('')
#     break

def get_czech_verse_corpus(number_of_books=20, order='first'):
    vocabulary = set()
    corpus = []
    for book in books:
        for poem_dict in book:
            for text in poem_dict['body']:
                for line in text:
                    tokens = line['text'].rstrip().lower().split()
                    if not tokens:
                        continue
                    if order == 'first':
                        tokens = ['<s>'] + tokens + ['</s>']
                    elif order == 'second':
                        tokens = ['<ss>'] + ['<s>'] + tokens + ['</s>']

                    vocabulary.update(tokens)
                    corpus.append(tokens)
    return vocabulary, corpus


In [31]:
vocabulary, corpus = get_czech_verse_corpus()
print(f"number of sentences: {len(corpus)}")
print(f"vocabulary size: {len(vocabulary)}")

model = BiGramModel(vocabulary=vocabulary, corpus=corpus, smooth=False)
# model_smoothed = BiGramModel(vocabulary=vocabulary, corpus=corpus, smooth=True)

number of sentences: 17946
vocabulary size: 26371


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: RuntimeWarning:

invalid value encountered in true_divide



In [32]:
sample_from_model(model)
print()
sample_from_model_top_p(model)
# print()
# sample_from_model(model_smoothed)
# print()
# sample_from_model_top_p(model_smoothed)

a listí chřestí:
píseň paprsk v chvíle zmocni se,
již v duši vězní hříchů říši.
ať bratí tvou zeje tmavý,

a na zem s námi se v srdci
jak v duši se v té písně myslím,
na zem se v tom světě
na chvíli


=======================================================================

(my notes)
http://seba1511.net/tutorials/beginner/nlp/sequence_models_tutorial.html